In [1]:
import requests
import re
from time import sleep
from bs4 import BeautifulSoup

domain = "https://www.kobe-np.co.jp"
startpath = "/news/keizai"

In [2]:
basesoup = BeautifulSoup(requests.get(domain + startpath).text, "lxml")

In [3]:
links = []
for a in basesoup.find_all("a", href=re.compile("/news/keizai/[0-9]+/.*")):
    links.append(domain + a.get("href"))

In [10]:
texts = []
for link in links[:3]:
    sleep(1)
    soup = BeautifulSoup(requests.get(link).text, "lxml")
    texts.append(soup.find("h4", class_="news-title").text.strip() + " " + soup.find("div", class_="news-contents").text.strip())

In [14]:
texts[0]

'神鋼子会社、新たにＪＩＳ違反\u3000２度目で認証取り消し 神戸製鋼所は１５日、グループ会社コベルコマテリアル銅管秦野工場（神奈川県秦野市）の銅管製品で、工業製品の品質や安全性の基準として国が定める日本工業規格（ＪＩＳ）の法令違反があり、認証を取り消されたと発表した。今年発覚したデータ改ざん問題に絡み、この工場でのＪＩＳ違反は１０月に続き２回目。民間団体である国際標準化機構（ＩＳＯ）の品質管理に関する国際規格の認証も取り消された。\u3000これらの規格は製品の性能や品質管理体制を保証し、取引の条件に指定する会社もある。認証の取り消しが広がれば製品の信用が低下し、販売に影響する可能性がある。\u3000アルミニウム製品を製造する大安製造所（三重県いなべ市）では、ＩＳＯの認証が一時停止されたことも判明した。品質管理の体制が不十分と判断された。\u3000ＪＩＳの認証機関は１５日までにグループ会社の神鋼メタルプロダクツ（北九州市）にも立ち入り審査を実施した。経済産業省は認証機関に対し、神戸製鋼の全認証工場への立ち入り審査を検討するよう指示している。\u3000秦野工場は、規格を満たさない製品にＪＩＳマークを付けて出荷していた。新たにＪＩＳ認証を取り消されたのは樹脂で表面を覆った銅管で、給湯器の配管に使われる。１０月には、エアコンに使われる銅管のＪＩＳ認証が取り消された。\u3000大安製造所は神戸製鋼のアルミ・銅事業部門で、今月１０日付で大安工場から名称が変わった。'